In [ ]:
!pip install pyspark

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('tree').getOrCreate()
df = spark.read.csv('/content/drive/MyDrive/Datasets/Thesis/dr16.csv', inferSchema=True, header=True)
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- objid: double (nullable = true)
 |-- modelMag_u: double (nullable = true)
 |-- modelMag_g: double (nullable = true)
 |-- modelMag_r: double (nullable = true)
 |-- modelMag_i: double (nullable = true)
 |-- modelMag_z: double (nullable = true)
 |-- fiberMag_u: double (nullable = true)
 |-- fiberMag_g: double (nullable = true)
 |-- fiberMag_r: double (nullable = true)
 |-- fiberMag_i: double (nullable = true)
 |-- fiberMag_z: double (nullable = true)
 |-- petroR50_r: double (nullable = true)
 |-- petroR90_r: double (nullable = true)
 |-- petroR50_z: double (nullable = true)
 |-- petroR90_z: double (nullable = true)
 |-- r: double (nullable = true)
 |-- i: double (nullable = true)
 |-- z: double (nullable = true)
 |-- redshift: double (nullable = true)
 |-- zerr: double (nullable = true)
 |-- mmug: double (nullable = true)
 |-- mmgr: double (nullable = true)
 |-- mmri: double (nullable = true)
 |-- mmiz: double (nullable = true)
 |-- mfug: doub

In [ ]:
import pyspark.sql.functions as func
df = df.withColumn("redshift", func.round(df["redshift"], 2).cast('integer'))

In [ ]:
import pandas as pd
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
_c0,0.000000e+00,1.000000e+00,2.000000e+00,3.000000e+00,4.000000e+00
objid,1.237680e+18,1.237680e+18,1.237680e+18,1.237680e+18,1.237680e+18
modelMag_u,2.163269e+01,1.974829e+01,2.374654e+01,2.063075e+01,2.138126e+01
modelMag_g,2.127911e+01,1.945819e+01,2.319651e+01,2.025426e+01,2.113488e+01
modelMag_r,2.122550e+01,1.933118e+01,2.201303e+01,2.004478e+01,2.109993e+01
modelMag_i,2.095690e+01,1.901371e+01,2.080744e+01,1.979378e+01,2.092882e+01
modelMag_z,2.082752e+01,1.880369e+01,1.991660e+01,1.970326e+01,2.080634e+01
fiberMag_u,2.205036e+01,2.010190e+01,2.460136e+01,2.099431e+01,2.185225e+01
fiberMag_g,2.162439e+01,1.979539e+01,2.364122e+01,2.060174e+01,2.145860e+01
fiberMag_r,2.164663e+01,1.966580e+01,2.248040e+01,2.040031e+01,2.145340e+01


In [ ]:
df.columns

['_c0',
 'objid',
 'modelMag_u',
 'modelMag_g',
 'modelMag_r',
 'modelMag_i',
 'modelMag_z',
 'fiberMag_u',
 'fiberMag_g',
 'fiberMag_r',
 'fiberMag_i',
 'fiberMag_z',
 'petroR50_r',
 'petroR90_r',
 'petroR50_z',
 'petroR90_z',
 'r',
 'i',
 'z',
 'redshift',
 'zerr',
 'mmug',
 'mmgr',
 'mmri',
 'mmiz',
 'mfug',
 'mfgr',
 'mfri',
 'mfiz']

In [ ]:
df.show(5)

+---+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|_c0|               objid|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|             zerr|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+---+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+-----------------+---------+------------------+---------

In [ ]:
df = df.drop('objid', '_c0', 'zerr')
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+---------+------------------+---------+--------+------------------+---------+--------+------------------+
|  21.63269|  21.27911|   21.2255|   20.9569|  20.82752|  22.

In [ ]:
#from pyspark.sql.functions import monotonically_increasing_id
#df = df.withColumn("objid", monotonically_increasing_id())
#df.show(5)

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz'], outputCol = 'features')

In [ ]:
output = assembler.transform(df)

In [ ]:
final_df = output.select('features', 'redshift')
final_df.show(3)

+--------------------+--------+
|            features|redshift|
+--------------------+--------+
|[21.63269,21.2791...|       1|
|[19.74829,19.4581...|       2|
|[23.74654,23.1965...|       0|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
layers = [25, 24, 24, 8]

In [ ]:
#trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
classifier = MultilayerPerceptronClassifier(labelCol='redshift', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)


In [ ]:
start_time = time.time()
model = classifier.fit(train)
print("%s seconds" % (time.time() - start_time))

In [ ]:
result = model.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Random Forest Accu:', multi_evaluator.evaluate(result))

Random Forest Accu: 0.8539784435309296


In [ ]:
precision_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'weightedPrecision')
print('Random Forest Accu:', precision_evaluator.evaluate(result))

Random Forest Accu: 0.8258246521231389


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(result))

test data (weightedPrecision):  0.8258246521231389


In [ ]:
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(result))

test data (weightedRecall):  0.8539784435309297


#Cascading

In [ ]:
result2 = model.transform(final_df)

In [ ]:
result2.show(3)

+--------------------+--------+--------------------+--------------------+----------+
|            features|redshift|       rawPrediction|         probability|prediction|
+--------------------+--------+--------------------+--------------------+----------+
|[21.63269,21.2791...|       1|[1.57716458308749...|[0.17029967441546...|       1.0|
|[19.74829,19.4581...|       2|[1.94432037178717...|[0.25368640497025...|       1.0|
|[23.74654,23.1965...|       0|[5.08391356729749...|[0.94851365831968...|       0.0|
+--------------------+--------+--------------------+--------------------+----------+
only showing top 3 rows



In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
df = df.withColumn("objid", monotonically_increasing_id())
df.show(5)

+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+---------+------------------+---------+--------+------------------+---------+--------+------------------+-----+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|petroR90_r|        petroR50_z|petroR90_z|       r|       i|       z|redshift|     mmug|              mmgr|     mmri|    mmiz|              mfug|     mfgr|    mfri|              mfiz|objid|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+----------+------------------+----------+--------+--------+--------+--------+---------+------------------+---------+--------+------------------+---------+--------+------------------+-----+
|  21.63269|  21.27911|   21.2255|   20.956

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
result2 = result2.withColumn("objid", monotonically_increasing_id())
result2.show(3)

+--------------------+--------+--------------------+--------------------+----------+-----+
|            features|redshift|       rawPrediction|         probability|prediction|objid|
+--------------------+--------+--------------------+--------------------+----------+-----+
|[21.63269,21.2791...|       1|[1.57716458308749...|[0.17029967441546...|       1.0|    0|
|[19.74829,19.4581...|       2|[1.94432037178717...|[0.25368640497025...|       1.0|    1|
|[23.74654,23.1965...|       0|[5.08391356729749...|[0.94851365831968...|       0.0|    2|
+--------------------+--------+--------------------+--------------------+----------+-----+
only showing top 3 rows



In [ ]:
print(df.count(), len(df.columns))
print(result2.count(), len(result2.columns))

new_df = df.join(result2, df.objid == result2.objid).select(df["*"],result2["prediction"])
#new_df = df.join(dt_predictions, df.objid == dt_predictions.objid)
new_df.show(5)

3524477 27
3524477 6
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+------------------+----------+------------------+--------+--------+--------+--------+---------+------------------+------------------+---------+---------+---------+---------+---------+-----+----------+
|modelMag_u|modelMag_g|modelMag_r|modelMag_i|modelMag_z|fiberMag_u|fiberMag_g|fiberMag_r|fiberMag_i|fiberMag_z|        petroR50_r|        petroR90_r|petroR50_z|        petroR90_z|       r|       i|       z|redshift|     mmug|              mmgr|              mmri|     mmiz|     mfug|     mfgr|     mfri|     mfiz|objid|prediction|
+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+------------------+------------------+----------+------------------+--------+--------+--------+--------+---------+------------------+------------------+---------+---------+---------+---------+---------+----

In [ ]:
assembler = VectorAssembler(inputCols = ['modelMag_u', 'modelMag_g', 'modelMag_r', 'modelMag_i', 'modelMag_z', 'fiberMag_u', 'fiberMag_g', 'fiberMag_r', 'fiberMag_i', 'fiberMag_z', 'petroR50_r', 'petroR90_r', 'petroR50_z', 'petroR90_z', 'r', 'i', 'z', 'mmug', 'mmgr', 'mmri', 'mmiz', 'mfug', 'mfgr', 'mfri', 'mfiz', 'prediction'], outputCol = 'features2')

In [ ]:
output = assembler.transform(new_df)

In [ ]:
final_df = output.select('features2', 'redshift')
final_df.show(3)

+--------------------+--------+
|           features2|redshift|
+--------------------+--------+
|[21.63269,21.2791...|       1|
|[20.90748,21.2373...|       1|
|[23.24831,21.2138...|       3|
+--------------------+--------+
only showing top 3 rows



In [ ]:
train, test = final_df.randomSplit([0.7, 0.3])

In [ ]:
layers = [26, 24, 24, 8]

In [ ]:
classifier2 = MultilayerPerceptronClassifier(labelCol='redshift', featuresCol='features2', maxIter=100, layers=layers, blockSize=128, seed=1234)

In [ ]:
model2 = classifier2.fit(train)

In [ ]:
result2 = model2.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'redshift', metricName = 'accuracy')
print('Decision Tree Accu:', multi_evaluator.evaluate(result2))

Decision Tree Accu: 0.8538496959634192


In [ ]:
print('test data (weightedPrecision): ', multi_evaluator.setMetricName('weightedPrecision').evaluate(result2))
print('test data (weightedRecall): ', multi_evaluator.setMetricName('weightedRecall').evaluate(result2))

test data (weightedPrecision):  0.8276868780281237
test data (weightedRecall):  0.8538496959634191
